<a href="https://colab.research.google.com/github/stevehooker/welsh-dragons/blob/main/notebooks/comfyui_colab_fast.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ComfyUI on Google Colab (FAST Edition)

**Version 3.1 - 31 Dec 2025**

**⚡ Target startup time: ~8-12 minutes** (down from 30-40 minutes)

## Key Optimisations

1. **Local Custom Nodes** - Copies custom_nodes to Colab's SSD (10x faster imports)
2. **Automatic Dependencies** - Uses ComfyUI-Manager's `restore-dependencies` to scan all nodes and install what's actually needed (no hardcoded list)

**How it works:**
- Models stay on Google Drive (too large to copy, ~50GB+)
- Custom nodes copied to fast local storage (~2-3 min copy)
- Dependencies auto-detected from each node's requirements.txt
- ComfyUI runs from hybrid setup (local code + Drive models)

**Setup Instructions:**
1. Enable GPU: Runtime → Change runtime type → T4 GPU
2. Add secrets (optional): 🔑 icon → Add API keys
3. Run all cells: Runtime → Run all
4. Wait for the cloudflared URL to appear

In [ ]:
#@title 0. Load Secrets (Optional)
import os

try:
    from google.colab import userdata
    secrets_found = []
    for key in ['ANTHROPIC_API_KEY', 'OPENAI_API_KEY', 'HF_TOKEN']:
        try:
            value = userdata.get(key)
            if value:
                os.environ[key] = value
                secrets_found.append(key)
        except:
            pass
    if secrets_found:
        print(f"✓ Loaded secrets: {', '.join(secrets_found)}")
    else:
        print("ℹ️  No secrets found. Add them in the 🔑 Secrets panel if needed.")
except Exception as e:
    print(f"ℹ️  Secrets not available: {e}")

In [ ]:
#@title 1. Mount Drive & Setup Paths
import os
import time

print("Mounting Google Drive...")
from google.colab import drive
drive.mount('/content/drive')

# Paths
DRIVE_COMFYUI = "/content/drive/MyDrive/ComfyUI"
LOCAL_COMFYUI = "/content/ComfyUI"

if not os.path.exists(DRIVE_COMFYUI):
    raise RuntimeError(f"❌ ComfyUI not found at {DRIVE_COMFYUI}")

print(f"✓ Drive workspace: {DRIVE_COMFYUI}")
print(f"✓ Local workspace: {LOCAL_COMFYUI}")

In [ ]:
#@title 2. Copy ComfyUI to Local Storage (The Key Optimisation)
#@markdown This copies ComfyUI core + custom_nodes to fast local SSD.
#@markdown Models stay on Drive (symlinked).

import shutil
import os
import time

start_time = time.time()

# Clean any previous setup
if os.path.exists(LOCAL_COMFYUI):
    print("Cleaning previous local copy...")
    shutil.rmtree(LOCAL_COMFYUI)

os.makedirs(LOCAL_COMFYUI, exist_ok=True)

print("=" * 60)
print("Copying ComfyUI to local storage (this is the speed boost!)")
print("=" * 60)

# List of items to copy locally (code, not models)
items_to_copy = [
    'main.py',
    'requirements.txt',
    'comfy',
    'comfy_extras',
    'custom_nodes',
    'web',
    'script_examples',
    'user',
    'notebooks',
    'app',
    '.git',  # Needed for version detection
]

# Additional single files
single_files = [
    'execution.py',
    'server.py', 
    'folder_paths.py',
    'nodes.py',
    'node_helpers.py',
    'latent_preview.py',
    'extra_model_paths.yaml',
]

for item in items_to_copy:
    src = os.path.join(DRIVE_COMFYUI, item)
    dst = os.path.join(LOCAL_COMFYUI, item)
    if os.path.exists(src):
        print(f"  Copying {item}...", end=" ", flush=True)
        t = time.time()
        if os.path.isdir(src):
            shutil.copytree(src, dst, symlinks=True)
        else:
            shutil.copy2(src, dst)
        print(f"({time.time()-t:.1f}s)")

for item in single_files:
    src = os.path.join(DRIVE_COMFYUI, item)
    dst = os.path.join(LOCAL_COMFYUI, item)
    if os.path.exists(src):
        shutil.copy2(src, dst)

# Symlink model directories (stay on Drive - too large to copy)
print("\nCreating symlinks for model directories...")
model_dirs = [
    'models',
    'input',
    'output',
]

for dir_name in model_dirs:
    src = os.path.join(DRIVE_COMFYUI, dir_name)
    dst = os.path.join(LOCAL_COMFYUI, dir_name)
    if os.path.exists(src) and not os.path.exists(dst):
        os.symlink(src, dst)
        print(f"  ✓ Symlinked {dir_name} → Drive")

elapsed = time.time() - start_time
print(f"\n✓ Local setup complete in {elapsed:.1f} seconds")
print(f"\nCustom nodes will now load from LOCAL SSD (10x+ faster!)")

In [ ]:
#@title 3. Install Dependencies (Auto-detected)
#@markdown Uses ComfyUI-Manager to scan all custom nodes and install their requirements.
#@markdown This is how your local install works - no hardcoded package list needed.

import time
start = time.time()

print("=" * 60)
print("Installing Dependencies")
print("=" * 60)

%cd {LOCAL_COMFYUI}

# 1. Core ComfyUI requirements
print("\n[1/3] Core ComfyUI requirements...")
!pip install -q -r requirements.txt
!pip install -q --upgrade comfyui-frontend-package

# 2. Use ComfyUI-Manager to auto-detect and install all custom node dependencies
# This is exactly what your local install does - scans each node's requirements.txt
print("\n[2/3] Scanning custom nodes for dependencies (cm-cli restore-dependencies)...")
print("       This reads requirements.txt from each custom node folder.")
!python custom_nodes/ComfyUI-Manager/cm-cli.py restore-dependencies

# 3. Upgrade frontend package (sometimes needed after restore-dependencies)
print("\n[3/3] Final cleanup...")
!pip install -q --upgrade comfyui-frontend-package

print(f"\n✓ All dependencies installed in {time.time()-start:.1f}s")

In [ ]:
#@title 4. Verify GPU
import torch

print(f"PyTorch {torch.__version__}")
if torch.cuda.is_available():
    print(f"CUDA {torch.version.cuda}")
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print("\n✓ GPU ready!")
else:
    print("\n❌ No GPU! Go to: Runtime > Change runtime type > T4 GPU")

In [ ]:
#@title 5. Start ComfyUI
import subprocess
import threading
import time
import socket
import re
import os

# Download cloudflared
![ ! -f ~/cloudflared-linux-amd64.deb ] && wget -q -O ~/cloudflared-linux-amd64.deb https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
!dpkg -i ~/cloudflared-linux-amd64.deb 2>/dev/null

cloudflared_url = None
comfyui_ready = False
start_time = time.time()

def start_tunnel(port=8188):
    """Start cloudflared and capture URL silently"""
    global cloudflared_url
    
    # Wait for ComfyUI server
    for _ in range(120):
        try:
            sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
            sock.settimeout(1)
            result = sock.connect_ex(('127.0.0.1', port))
            sock.close()
            if result == 0:
                break
        except:
            pass
        time.sleep(0.5)
    
    # Start cloudflared
    proc = subprocess.Popen(
        ["cloudflared", "tunnel", "--url", f"http://127.0.0.1:{port}"],
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE,
        text=True,
        bufsize=1
    )
    
    for line in proc.stderr:
        if "trycloudflare.com" in line:
            match = re.search(r'https://[a-z0-9-]+\.trycloudflare\.com', line)
            if match:
                cloudflared_url = match.group(0)
                break

# Start tunnel in background
tunnel_thread = threading.Thread(target=start_tunnel, daemon=True)
tunnel_thread.start()

# Start ComfyUI from LOCAL directory
%cd {LOCAL_COMFYUI}

print("=" * 70)
print("Starting ComfyUI from LOCAL storage (custom nodes load 10x faster!)")
print("=" * 70)
print()

proc = subprocess.Popen(
    ["python", "main.py", "--dont-print-server"],
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT,
    text=True,
    bufsize=1
)

for line in proc.stdout:
    print(line, end='')
    
    # Detect when ready
    if "Import times for custom nodes:" in line and not comfyui_ready:
        comfyui_ready = True
        time.sleep(2)
        
        elapsed = time.time() - start_time
        
        if cloudflared_url:
            print("\n" + "=" * 70)
            print(f"\n🎉 COMFYUI READY in {elapsed:.0f} seconds!")
            print(f"\n🌐 ACCESS HERE: {cloudflared_url}")
            print("\n" + "=" * 70 + "\n")
        else:
            print("\n⚠️  Waiting for tunnel URL...")
            for _ in range(15):
                if cloudflared_url:
                    print("\n" + "=" * 70)
                    print(f"\n🎉 COMFYUI READY in {elapsed:.0f} seconds!")
                    print(f"\n🌐 ACCESS HERE: {cloudflared_url}")
                    print("\n" + "=" * 70 + "\n")
                    break
                time.sleep(1)

## How This Works

### The Local-Style Dependency System

Your local ComfyUI doesn't need a hardcoded list of packages because:

1. **ComfyUI-Manager scans** each folder in `custom_nodes/`
2. **Reads** each node's `requirements.txt` (or `install.py`)
3. **Compares** against installed packages
4. **Installs** only what's missing

The command `cm-cli.py restore-dependencies` does exactly this.

### Why the old Colab took 30-40 minutes

When `restore-dependencies` ran from Google Drive, it had to:
- Read hundreds of small files (each with ~100-500ms latency)
- Run pip checks (which also hit the filesystem)
- The I/O overhead dominated everything

### Why this version is faster

1. **Copy to local first** (~2-4 min) - one bulk transfer
2. **restore-dependencies on local SSD** (~3-5 min) - fast file access
3. **Node loading from local** (~2-3 min) - 10x faster imports

**Total: ~8-12 min** vs **30-40 min** before

### Trade-offs
- ✓ Automatic - no need to maintain a package list
- ✓ Always matches your actual custom nodes
- ✓ Much faster node loading
- ✗ Copy step adds ~2-4 min upfront (but saves 20+ min later)
- ✗ Changes to custom_nodes on Drive need notebook restart